# Using pywbem mock to create a simple repository

The pywbem mock package (pywbem_mock) allows a user of the pywbem to mock a WBEM server so that pywbem WBEM request methods can be tested without having a WBEM server available;

The pywbem mock support consists of the pywbem_mock.FakedWBEMConnection class that establishes a faked connection. That class is a subclass of pywbem.WBEMConnection and replaces its internal methods that use HTTP/HTTPS to communicate with a WBEM server with methods that instead operate on a local in-memory repository of CIM objects (the mock repository).

As a result, the operation methods of FakedWBEMConnection are those inherited from WBEMConnection, so they have the exact same input parameters, output parameters, return values, and even most of the raised exceptions, as when being invoked on a WBEMConnection object against a WBEM server.

Each FakedWBEMConnection object has its own mock repository. The mock repository contains the same kinds of CIM objects a WBEM server repository contains: CIM classes, CIM instances, and CIM qualifier types (declarations), all contained in CIM namespaces.

Because FakedWBEMConnection operates only on the mock repository, the class does not have any connection- or security-related constructor parameters.

FakedWBEMConnection has some additional methods that provide for adding CIM classes, instances and qualifier types to its mock repository. 

Instances in the repository can be modified or deleted by using operation methods such as DeleteClass() or ModifyInstance(). Classes and qualifiers can be deleted using the corresponding delete methods.

The following code demonstrates adding a simple set of qualifier declarations and a class to the repository
by compiling their MOF definition.

In [1]:
import pywbem
import pywbem_mock

# MOF string defining qualifiers declarations, class, and instance
# This mof will be used throughout this notebook
mof = '''
    Qualifier Key : boolean = false,
        Scope(property, reference),
        Flavor(DisableOverride, ToSubclass);
    Qualifier Description : string = null,
        Scope(any),
        Flavor(EnableOverride, ToSubclass, Translatable);    
    Qualifier In : boolean = true, 
        Scope(parameter), 
        Flavor(DisableOverride, ToSubclass);

         [Description ("This is a dumb test class")]
    class CIM_Foo {
            [Key, Description ("This is key prop")]
        string InstanceID;
            [Description ("This is some simplistic data")]
        Uint32 SomeData;
            [Description ("This is a method without parameters")]
        string Method1();
            [Description ("This is a second method with parameter")]
        uint32 Delete([IN, Description("blahblah")]
          boolean Immediate);
    };

    instance of CIM_Foo as $I1 { InstanceID = "I1"; SomeData=3; };
    '''

# Create a faked connection (with a mock repository in full mode)
conn = pywbem_mock.FakedWBEMConnection(default_namespace='root/cimv2')

# Compile the MOF string and add its CIM objects to the default namespace
# of the mock repository
conn.compile_mof_string(mof)

## Display the created repository

At any time, the data in the repository can be displayed using the method display_repository().

In [2]:
conn.display_repository()

# ========Mock Repo Display fmt=mof namespaces=all =========


# NAMESPACE 'root/cimv2'

# Namespace 'root/cimv2': contains 3 Qualifier Declarations 

Qualifier Description : string,
    Scope(any),
    Flavor(EnableOverride, ToSubclass, Translatable);

Qualifier In : boolean = true,
    Scope(parameter),
    Flavor(DisableOverride, ToSubclass);

Qualifier Key : boolean = false,
    Scope(property, reference),
    Flavor(DisableOverride, ToSubclass);

# Namespace 'root/cimv2': contains 1 Classes 

   [Description ( "This is a dumb test class" )]
class CIM_Foo {

      [Key ( true ),
       Description ( "This is key prop" )]
   string InstanceID;

      [Description ( "This is some simplistic data" )]
   uint32 SomeData;

      [Description ( "This is a method without parameters" )]
   string Method1();

      [Description ( "This is a second method with parameter" )]
   uint32 Delete(
         [IN ( true ),
          Description ( "blahblah" )]
      boolean Immediate);

};

# Namespa

## Execute simple operations on the mock repository

Once qualifier declarations, classes, and instances have been inserted into  the mock repository they
can be retrieved using the WBEMConnection methods provided by pyebem.

Thus, WBEMConnection.getQualifier() retrieves a single qualifier declaration. The method tomof() is
a method in pywbem cimobject classes CIMQualifierDeclaration, CIMClass, and CIMInstance and is an easy
way to display the objects returned from the repository

In [3]:
from pywbem import CIMInstanceName, Error
# Perform operations on the faked connection:

# Enumerate top-level classes in the default namespace (without subclasses)
classes = conn.EnumerateClasses();

# Get the 'Description' qualifier type in the default namespace
qd = conn.GetQualifier('Description')
print(qd.tomof())

# Enumerate subclasses of 'CIM_Foo' in the default namespace (without subclasses)
classes = conn.EnumerateClasses(classname='CIM_Foo')
for cls in classes:
    print(cls.tomof())

# Get 'CIM_Foo' class in the default namespace
my_class = conn.GetClass('CIM_Foo')

# Get a specific instance of 'CIM_Foo' in the default namespace
keybindings = {'InstanceID': "I1"}
inst = conn.GetInstance(CIMInstanceName('CIM_Foo', keybindings))
print(inst.tomof())
# print the path of the returned instance
print("path:%s" % inst.path)

Qualifier Description : string,
    Scope(any),
    Flavor(EnableOverride, ToSubclass, Translatable);

class CIM_Foo {

   string InstanceID;

   uint32 SomeData;

   string Method1();

   uint32 Delete(
      boolean Immediate);

};

instance of CIM_Foo {
   InstanceID = "I1";
   SomeData = 3;
};

path:root/cimv2:CIM_Foo.InstanceID="I1"


## Create a new instance of the defined class

Creating an instance primarily involves attaching properties with their name, value (and often type)
as a dictionary to a CIMInstance with the  name of the class for the new instance. The method
WBEMConnection.CreateInstance only requires the new instance object (and the namespace if the default
namespace is not being used) to manage the instance that is being created.

The mocker simple creates the path for this instance and inserts the new instance into the mock
repository. Note that a successful CreateInstance returns a CIMInstanceName for the new instance.

In [4]:
from pywbem import CIMInstance, Uint32

p = {"InstanceID": "I2", "SomeData": Uint32(999)}

newinst = CIMInstance("CIM_Foo", properties=p)
new_path = None
try:
    new_path = conn.CreateInstance(newinst)
    print("Return path: %s" % new_path)
except Error as er:
    print("Exception on CreateInstance. exception=%s" % er)


Return path: root/cimv2:CIM_Foo.InstanceID="I2"


## Now retrieve the new instance from the mock server

Here we retrieve the instance we just created from the existing connection and display the
instance using the tomof() method.

In [5]:
myinst = conn.GetInstance(CIMInstanceName('CIM_Foo', keybindings={'InstanceID': "I2"}))

print("path:%s\n%s" % (myinst.path, myinst.tomof()))

path:root/cimv2:CIM_Foo.InstanceID="I2"
instance of CIM_Foo {
   SomeData = 999;
   InstanceID = "I2";
};



## Get the new instance with the path returned by the create

Since the CreateInstance returned the path the server created for the instance it inserted into
ins repository, that path can also be used to retrieve the instance from the repository

In [6]:
myinst2 = conn.GetInstance(new_path)
print(myinst2.tomof())


instance of CIM_Foo {
   SomeData = 999;
   InstanceID = "I2";
};



# We can also retrieve all instances of that class

In this case we display the returned instances using the string representation

In [7]:
insts = conn.EnumerateInstances("CIM_Foo")
for inst in insts:
    print("path=%s" % inst.path)
    print("%s" % inst)
 

path=root/cimv2:CIM_Foo.InstanceID="I1"
CIMInstance(classname='CIM_Foo', path=CIMInstanceName(classname='CIM_Foo', keybindings=NocaseDict({'InstanceID': 'I1'}), namespace='root/cimv2', host=None), ...)
path=root/cimv2:CIM_Foo.InstanceID="I2"
CIMInstance(classname='CIM_Foo', path=CIMInstanceName(classname='CIM_Foo', keybindings=NocaseDict({'InstanceID': 'I2'}), namespace='root/cimv2', host=None), ...)


# Finally we can delete the new instance


In [8]:
try:
    conn.DeleteInstance(new_path)
    for inst in insts:
        print("path=%s" % inst.path)
except Error as er:
    print("Error with delete")
    

path=root/cimv2:CIM_Foo.InstanceID="I1"
path=root/cimv2:CIM_Foo.InstanceID="I2"


## Executing a method

The mocker can executed CIM methods as if they were on a server.  However, since there are no real providers
in the mocker, the user must define what a method would do.  This definition is provided to the mocker
by defining a callback and installing it with the add_method_callback method.

In [14]:
from pywbem import CIMParameter

# Definition of callback method. This must be registered to
# the mock environment with add_method_callback
def method1_callback(conn, methodname, objectname, **params):
    """
    Callback function that demonstrates what can be done, without being
    really useful.
    """

    # Access input parameters
    print('Callback received params: %r' % params )
    print('Callback recieved object_name %s' % objectname)
    ip1 = params['InputParam1']

    # Access the mock repository through the faked connection object.
    cl = conn.GetClass(objectname)

    # Set return value == 0 and output parameters
    rtn_val = 0
    # setup output parameter that will be returned to caller
    op1 = CIMParameter('OutputParam11', 'string', value='Some output data')
    return rtn_val, [op1]

more_mof = '''
    Qualifier Out : boolean = false,
        Scope(parameter),
        Flavor(DisableOverride, ToSubclass);

    Qualifier Static : boolean = false, 
        Scope(property, method), 
        Flavor(DisableOverride, ToSubclass);

    class TST_Class {

        string InstanceID;

          [Static,
           Description("Static method with input and output parameters")]
        uint32 Method1(
            [IN, Description("Input param 1")]
          string InputParam1,
            [IN (False), OUT, Description("Output param 1")]
          string OutputParam1);
    };
'''

# Compile the MOF string and add its CIM objects to the default namespace
# of the mock repository
conn.compile_mof_string(more_mof)

# Register the method callback function to the mock repository, for the
# default namespace of the connection

# try block allows excuting cell multiple times without exception
# because method already registered.
try:
    conn.add_method_callback('TST_Class', 'Method1', method1_callback)
except ValueError as ve:
    print(ve)


# Define a value for the Method Parameter IP1
params = [('InputParam1', 'someData')]
 # Invoke static method Method1
result = conn.InvokeMethod('Method1', 'TST_Class', Params=params)

print('Return value: %r' % result[0])
print('Output parameters: %r' % (result[1],))

Duplicate method specification
params: {'InputParam1': CIMParameter(name='InputParam1', type='string', reference_class=None, is_array=False, array_size=None, qualifiers=NocaseDict({}), value='someData', embedded_object=None)}
object_name root/cimv2:TST_Class
Return value: 0
Output parameters: NocaseDict({'OutputParam11': 'Some output data'})
